# NAFION 10.1063/1.4894813 MODEL 

## LAMMPS PARAMETERS FILES

Polymer: 3 reps. C$_{65}$H$_{3}$F$_{129}$O$_{15}$S$_{3}$.

Monomer: C$_{23}$HF$_{47}$O$_5$S	

<img src="model.png" alt="model" width="400"/>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='ticks')

import re
import os

def get_file(filename):
    """
    returns raw dataframe from any type of file
    """
    with open(filename, "r") as file:
        file_content = file.readlines()
    df = pd.DataFrame(file_content, columns=['data'])
    return df

In [2]:
! ls

 ATB					       NAFION_3repsPOLYMER.data
'generate LAMMPS files for C23HF47O5S.ipynb'  'Other Chains'
 model.png				       polymer_3reps.mol


In [3]:
#! code NAFION_3repsPOLYMER.data

In [4]:
data = get_file("NAFION_3repsPOLYMER.data")
data = data.loc[146:360]
data["data"] = data["data"].str.split('\s')
data["data"] = data["data"].apply(lambda x: list(filter(None, x)))
data["atom id"] = data["data"].apply(lambda x: x[0]).astype(int)
data["element id"] = data["data"].apply(lambda x: x[2]).astype(int)
data["ATB charge"] = data["data"].apply(lambda x: x[3]).astype(float)
data["x"] = data["data"].apply(lambda x: x[4]).astype(float)
data["y"] = data["data"].apply(lambda x: x[5]).astype(float)
data["z"] = data["data"].apply(lambda x: x[6]).astype(float)
data["elements"] = data["element id"].map({1:"O", 2:"O", 3:"C", 
                        4:"H", 5:"F", 6:"S", 7:"H", 8:"O", 9:"C"})
data.reset_index(drop=True,inplace=True)

In [5]:
# Created the following dictionary by looking at ovito and the charges model

# main chain:
C_ends =  [2,190] #0.5769
C_charges_1stmCHAIN = list(np.arange(2,45,3)[1:]) # 0.3846
C_OwM = [47,116,185] # Carbon connected to oxygen with main chains 0.3218
F_charges_1stmCHAIN = [i for i in list(range(1,49)) if i not in C_ends + C_OwM + C_charges_1stmCHAIN] # -0.1923
C_side2CowM = [71,140,187] # 0.3846 | the carbon that connects the reps
C_charges_2ndmCHAIN = list(np.arange(74,114,3)) # 0.3846
F_charges_2ndmCHAIN = [i for i in list(range(72,118)) if i not in C_charges_2ndmCHAIN + C_OwM]  # -0.1923
C_charges_3rdmCHAIN = list(np.arange(143,183,3)) # 0.3846
F_charges_3rdmCHAIN = [i for i in list(range(141,194)) if i 
                       not in C_charges_3rdmCHAIN + C_ends + C_side2CowM + C_OwM]  # -0.1923

C_main_chain = C_charges_1stmCHAIN + C_charges_2ndmCHAIN + C_charges_3rdmCHAIN
F_main_chain = F_charges_1stmCHAIN + F_charges_2ndmCHAIN + F_charges_3rdmCHAIN

# side chain:
O_1sChain = [49, 118, 194] # -0.2742
O_2sChain = [59, 128, 204] # -0.2604

end_C_sideChain = [55, 124, 200] # 0.4947
end_F_sideChain = [56,57,58,125,126,127,201,202,203] #-0.1649

S = [66,135,211] # 1.4124
O_boundded_S = [67,68,69,136,137,138,212,213,214] # -0.6320
H = [70,139,215] # 0

# specifics side chain:
C_4010 = [53,122,198] # 0.4010
F_bounded_C4010 = [54,123,199] # -0.1913
C_3228 = [50,119,195] # 0.3228
F_bounded_C3228 = [51,52,120,121,196,197] # -0.1631
C_3218 = [60,129,205] # 0.3218
C_3216 = [63,132,208] # 0.3216
F_bounded_C3218 = [61,62,130,131,206,207] # -0.1662
F_bounded_C3216 = [64,65,133,134,209,210] # -0.3278

In [6]:
# Sanity Checks on main chain!
print( len(F_charges_1stmCHAIN[3:-1]) == len(F_charges_2ndmCHAIN[2:-1]) )
print( len(F_charges_2ndmCHAIN[2:-1]) == len(F_charges_3rdmCHAIN[2:-6]) )

True
True


In [7]:
list_id = C_ends + C_main_chain + F_main_chain + C_OwM + C_side2CowM + O_1sChain + O_2sChain + \
            end_C_sideChain + end_F_sideChain + C_4010 + F_bounded_C4010 + C_3228 + F_bounded_C3228 + \
            C_3218 + C_3216 + F_bounded_C3218 + F_bounded_C3216 + S + O_boundded_S + H

list_charges = [0.5769]*len(C_ends) + [0.3846]*len(C_main_chain) + [-0.1923]*len(F_main_chain) + \
               [0.3218]*len(C_OwM)  + [0.3846]*len(C_side2CowM)  + [-0.2742]*len(O_1sChain) + \
               [-0.2604]*len(O_2sChain) + [0.4947]*len(end_C_sideChain) + [-0.1649]*len(end_F_sideChain) + \
               [0.4010]*len(C_4010) + [-0.1913]*len(F_bounded_C4010) + \
               [0.3228]*len(C_3228) + [-0.1631]*len(F_bounded_C3228) + \
               [0.3218]*len(C_3218) + [0.3216]*len(C_3216) + [-0.1662]*len(F_bounded_C3218) + \
               [-0.3278]*len(F_bounded_C3216) + [1.4124]*len(S) + [-0.6320]*len(O_boundded_S) + [0]*len(H)

dct_charges = dict( zip(list_id, list_charges))

In [8]:
# Sanity check
len(dct_charges) == len(data)

True

In [9]:
data["charge"] = data["atom id"].map(dct_charges)

In [10]:
data["ATB charge"].sum()

3.469446951953614e-16

In [11]:
data["charge"].sum()

-3.080999999999999

# WRITE `.mol` FILE WITH CORRECT PARAMS

In [12]:
data

,data,atom id,element id,ATB charge,x,y,z,elements,charge
0,"[38, 1, 9, 0.212, -15.848633748637825, -0.3757...",38,9,0.212,-15.848634,-0.375800,-2.583642,C,0.3846
1,"[39, 1, 5, -0.104, -16.092463723142508, -0.354...",39,5,-0.104,-16.092464,-0.354311,-1.239281,F,-0.1923
2,"[40, 1, 5, -0.109, -16.51094827098748, -1.4924...",40,5,-0.109,-16.510948,-1.492468,-3.020010,F,-0.1923
3,"[41, 1, 9, 0.211, -14.272647810968664, -0.5007...",41,9,0.211,-14.272648,-0.500716,-2.901806,C,0.3846
4,"[42, 1, 5, -0.11, -14.070789048300265, -0.5632...",42,5,-0.110,-14.070789,-0.563213,-4.254946,F,-0.1923
...,...,...,...,...,...,...,...,...,...
210,"[213, 1, 1, -0.847, 20.310181874581446, 2.4904...",213,1,-0.847,20.310182,2.490435,1.488110,O,-0.6320
211,"[214, 1, 8, -0.757, 18.03607807409614, 1.96090...",214,8,-0.757,18.036078,1.960909,0.914882,O,-0.6320
212,"[215, 1, 7, 0.325, 17.09466190985118, 1.927366...",215,7,0.325,17.094662,1.927367,1.110210,H,0.0000
213,"[9, 1, 5, -0.105, -15.472128258843753, 10.2124...",9,5,-0.105,-15.472128,10.212485,2.686603,F,-0.1923


In [13]:
def get_bonds(polymer,slices):
    data = polymer.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    data["data"] = data["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    data["bond id"] = data["data"].apply(lambda x: x[0]).astype(int)
    data["bond type"] = data["data"].apply(lambda x: x[1]).astype(int)
    data["atom1"] = data["data"].apply(lambda x: x[2]).astype(int)
    data["atom2"] = data["data"].apply(lambda x: x[3]).astype(int)
    return data

def get_angles(data, slices):
    angles = data.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    angles["data"] = angles["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    angles["angle id"] = angles["data"].apply(lambda x: x[0]).astype(int)
    angles["angle type"] = angles["data"].apply(lambda x: x[1]).astype(int)
    angles["atom1"] = angles["data"].apply(lambda x: x[2]).astype(int)
    angles["atom2"] = angles["data"].apply(lambda x: x[3]).astype(int)
    angles["atom3"] = angles["data"].apply(lambda x: x[4]).astype(int)
    return angles


def get_dihedrals(data,slices):
    dihedrals = data.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    dihedrals["data"] = dihedrals["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    dihedrals["dihedral id"] = dihedrals["data"].apply(lambda x: x[0]).astype(int)
    dihedrals["dihedral type"] = dihedrals["data"].apply(lambda x: x[1]).astype(int)
    dihedrals["atom1"] = dihedrals["data"].apply(lambda x: x[2]).astype(int)
    dihedrals["atom2"] = dihedrals["data"].apply(lambda x: x[3]).astype(int)
    dihedrals["atom3"] = dihedrals["data"].apply(lambda x: x[4]).astype(int)
    dihedrals["atom4"] = dihedrals["data"].apply(lambda x: x[5]).astype(int)
    return dihedrals

def get_impropers(data,slices):
    impropers = data.loc[slices[0]:slices[1]].copy().reset_index(drop=True)
    impropers["data"] = impropers["data"].str.split('\s').apply(lambda x: list(filter(None, x)))
    impropers["improper id"] = impropers["data"].apply(lambda x: x[0]).astype(int)
    impropers["improper type"] = impropers["data"].apply(lambda x: x[1]).astype(int)
    impropers["atom1"] = impropers["data"].apply(lambda x: x[2]).astype(int)
    impropers["atom2"] = impropers["data"].apply(lambda x: x[3]).astype(int)
    impropers["atom3"] = impropers["data"].apply(lambda x: x[4]).astype(int)
    impropers["atom4"] = impropers["data"].apply(lambda x: x[5]).astype(int)
    return impropers


bonds     = get_bonds(get_file("NAFION_3repsPOLYMER.data"),[582,795])
angles    = get_angles(get_file("NAFION_3repsPOLYMER.data"),[799,1215])
dihedrals = get_dihedrals(get_file("NAFION_3repsPOLYMER.data"),[1219,1812])
#impropers = get_impropers(get_file("NAFION_3repsPOLYMER.data"),[1816,1816]) # I think this results from H error

In [14]:
# correct for H, remove element 4 y recorre el conteo de ids.
data.loc[data[data["element id"] >= 4].index, "element id"] = data[data["element id"] >= 4]["element id"] - 1

In [15]:
# remove Hydrogens
data[data["elements"].isin(["H"])]

,data,atom id,element id,ATB charge,x,y,z,elements,charge
109,"[139, 1, 7, 0.314, 0.8174042283034582, -10.826...",139,6,0.314,0.817404,-10.826816,4.543877,H,0.0
177,"[70, 1, 7, 0.325, -15.547969786114017, 4.73216...",70,6,0.325,-15.547970,4.732161,2.218391,H,0.0
212,"[215, 1, 7, 0.325, 17.09466190985118, 1.927366...",215,6,0.325,17.094662,1.927367,1.110210,H,0.0


In [16]:
data

,data,atom id,element id,ATB charge,x,y,z,elements,charge
0,"[38, 1, 9, 0.212, -15.848633748637825, -0.3757...",38,8,0.212,-15.848634,-0.375800,-2.583642,C,0.3846
1,"[39, 1, 5, -0.104, -16.092463723142508, -0.354...",39,4,-0.104,-16.092464,-0.354311,-1.239281,F,-0.1923
2,"[40, 1, 5, -0.109, -16.51094827098748, -1.4924...",40,4,-0.109,-16.510948,-1.492468,-3.020010,F,-0.1923
3,"[41, 1, 9, 0.211, -14.272647810968664, -0.5007...",41,8,0.211,-14.272648,-0.500716,-2.901806,C,0.3846
4,"[42, 1, 5, -0.11, -14.070789048300265, -0.5632...",42,4,-0.110,-14.070789,-0.563213,-4.254946,F,-0.1923
...,...,...,...,...,...,...,...,...,...
210,"[213, 1, 1, -0.847, 20.310181874581446, 2.4904...",213,1,-0.847,20.310182,2.490435,1.488110,O,-0.6320
211,"[214, 1, 8, -0.757, 18.03607807409614, 1.96090...",214,7,-0.757,18.036078,1.960909,0.914882,O,-0.6320
212,"[215, 1, 7, 0.325, 17.09466190985118, 1.927366...",215,6,0.325,17.094662,1.927367,1.110210,H,0.0000
213,"[9, 1, 5, -0.105, -15.472128258843753, 10.2124...",9,4,-0.105,-15.472128,10.212485,2.686603,F,-0.1923


In [17]:
# CELDA QUE SOLO SE PUEDE CORRER UNA VEZ
data = data[~(data["atom id"] == 70)]
data = data.sort_values("atom id").reset_index(drop=True)
data.loc[data[data["atom id"] >= 70].index, "atom id"] = data[data["atom id"] >= 70]["atom id"] - 1

data = data[~(data["atom id"] == 139 -1 )]
data = data.sort_values("atom id").reset_index(drop=True)
data.loc[data[data["atom id"] >= 139 -1 ].index, "atom id"] = data[data["atom id"] >= 139 -1 ]["atom id"] - 1

data = data[~(data["atom id"] == 215 - 2)]
data = data.sort_values("atom id").reset_index(drop=True)
data.loc[data[data["atom id"] >= 215 - 2].index, "atom id"] = data[data["atom id"] >= 215 - 2]["atom id"] - 1

In [18]:
data["elements"].unique()

array(['F', 'C', 'O', 'S'], dtype=object)

In [19]:
data["element id"].unique(), len(bonds)

(array([4, 8, 3, 2, 5, 1, 7]), 214)

In [20]:
bonds[bonds["atom2"].isin([70,139,215])]

,data,bond id,bond type,atom1,atom2
107,"[108, 9, 138, 139]",108,9,138,139
179,"[180, 9, 69, 70]",180,9,69,70
213,"[214, 9, 214, 215]",214,9,214,215


In [21]:
# CELDA QUE SOLO SE PUEDE CORRER UNA VEZ
bonds = bonds[~(bonds["atom2"].isin([70,139,215]))]
bonds = bonds.sort_values("bond id").reset_index(drop=True)
bonds.loc[bonds[bonds["atom1"] >= 70].index, "atom1"] = bonds[bonds["atom1"] >= 70]["atom1"] - 1
bonds.loc[bonds[bonds["atom2"] >= 70].index, "atom2"] = bonds[bonds["atom2"] >= 70]["atom2"] - 1

bonds.loc[bonds[bonds["atom1"] >= 139-1].index, "atom1"] = bonds[bonds["atom1"] >= 139-1]["atom1"] - 1
bonds.loc[bonds[bonds["atom2"] >= 139-1].index, "atom2"] = bonds[bonds["atom2"] >= 139-1]["atom2"] - 1

bonds.loc[bonds[bonds["atom1"] >= 215-2].index, "atom1"] = bonds[bonds["atom1"] >= 215-2]["atom1"] - 1
bonds.loc[bonds[bonds["atom2"] >= 215-2].index, "atom2"] = bonds[bonds["atom2"] >= 215-2]["atom2"] - 1

bonds.loc[bonds.index,"bond id"] = list(range(1,len(bonds)+1)) 

In [22]:
len(bonds)

211

In [23]:
angles[(angles["atom3"].isin([70,139,215]))] # 417 total of angles

,data,angle id,angle type,atom1,atom2,atom3
212,"[213, 26, 135, 138, 139]",213,26,135,138,139
353,"[354, 16, 66, 69, 70]",354,16,66,69,70
416,"[417, 16, 211, 214, 215]",417,16,211,214,215


In [24]:
# SOLO SE PUEDE CORRER UNA VEZ
angles = angles[~(angles["atom3"].isin([70,139,215]))]
angles = angles.sort_values("angle id").reset_index(drop=True)

angles.loc[angles[angles["atom1"] >= 70].index, "atom1"] = angles[angles["atom1"] >= 70]["atom1"] - 1
angles.loc[angles[angles["atom2"] >= 70].index, "atom2"] = angles[angles["atom2"] >= 70]["atom2"] - 1
angles.loc[angles[angles["atom3"] >= 70].index, "atom3"] = angles[angles["atom3"] >= 70]["atom3"] - 1

angles.loc[angles[angles["atom1"] >= 139-1].index, "atom1"] = angles[angles["atom1"] >= 139-1]["atom1"] - 1
angles.loc[angles[angles["atom2"] >= 139-1].index, "atom2"] = angles[angles["atom2"] >= 139-1]["atom2"] - 1
angles.loc[angles[angles["atom3"] >= 139-1].index, "atom3"] = angles[angles["atom3"] >= 139-1]["atom3"] - 1

angles.loc[angles[angles["atom1"] >= 215-2].index, "atom1"] = angles[angles["atom1"] >= 215-2]["atom1"] - 1
angles.loc[angles[angles["atom2"] >= 215-2].index, "atom2"] = angles[angles["atom2"] >= 215-2]["atom2"] - 1
angles.loc[angles[angles["atom3"] >= 215-2].index, "atom3"] = angles[angles["atom3"] >= 215-2]["atom3"] - 1

angles.loc[angles.index,"angle id"] = list(range(1,len(angles)+1)) 

In [25]:
angles

,data,angle id,angle type,atom1,atom2,atom3
0,"[1, 23, 35, 38, 39]",1,23,35,38,39
1,"[2, 27, 35, 38, 40]",2,27,35,38,40
2,"[3, 9, 35, 38, 41]",3,9,35,38,41
3,"[4, 4, 39, 38, 40]",4,4,39,38,40
4,"[5, 24, 39, 38, 41]",5,24,39,38,41
...,...,...,...,...,...,...
409,"[412, 22, 208, 211, 213]",410,22,206,209,211
410,"[413, 20, 208, 211, 214]",411,20,206,209,212
411,"[414, 31, 212, 211, 213]",412,31,210,209,211
412,"[415, 18, 212, 211, 214]",413,18,210,209,212


In [26]:
sorted(angles["angle type"].unique())

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33]

In [27]:
dihedrals[(dihedrals["atom4"].isin([70,139,215]))] # len dihedrals = 594 (-9 = 585)

,data,dihedral id,dihedral type,atom1,atom2,atom3,atom4
303,"[304, 1, 132, 135, 138, 139]",304,1,132,135,138,139
304,"[305, 6, 136, 135, 138, 139]",305,6,136,135,138,139
305,"[306, 6, 137, 135, 138, 139]",306,6,137,135,138,139
507,"[508, 1, 63, 66, 69, 70]",508,1,63,66,69,70
508,"[509, 6, 67, 66, 69, 70]",509,6,67,66,69,70
509,"[510, 6, 68, 66, 69, 70]",510,6,68,66,69,70
591,"[592, 1, 208, 211, 214, 215]",592,1,208,211,214,215
592,"[593, 6, 212, 211, 214, 215]",593,6,212,211,214,215
593,"[594, 6, 213, 211, 214, 215]",594,6,213,211,214,215


In [28]:
# SOLO SE PUEDE CORRER UNA VEZ
dihedrals = dihedrals[~(dihedrals["atom4"].isin([70,139,215]))]
dihedrals = dihedrals.sort_values("dihedral id").reset_index(drop=True)

dihedrals.loc[dihedrals[dihedrals["atom1"] >= 70].index, "atom1"] = dihedrals[dihedrals["atom1"] >= 70]["atom1"] - 1
dihedrals.loc[dihedrals[dihedrals["atom2"] >= 70].index, "atom2"] = dihedrals[dihedrals["atom2"] >= 70]["atom2"] - 1
dihedrals.loc[dihedrals[dihedrals["atom3"] >= 70].index, "atom3"] = dihedrals[dihedrals["atom3"] >= 70]["atom2"] - 1
dihedrals.loc[dihedrals[dihedrals["atom4"] >= 70].index, "atom4"] = dihedrals[dihedrals["atom4"] >= 70]["atom4"] - 1

dihedrals.loc[dihedrals[dihedrals["atom1"] >= 139-1].index, "atom1"] = dihedrals[dihedrals["atom1"] >= 139-1]["atom1"] - 1
dihedrals.loc[dihedrals[dihedrals["atom2"] >= 139-1].index, "atom2"] = dihedrals[dihedrals["atom2"] >= 139-1]["atom2"] - 1
dihedrals.loc[dihedrals[dihedrals["atom3"] >= 139-1].index, "atom3"] = dihedrals[dihedrals["atom3"] >= 139-1]["atom2"] - 1
dihedrals.loc[dihedrals[dihedrals["atom4"] >= 139-1].index, "atom4"] = dihedrals[dihedrals["atom4"] >= 139-1]["atom4"] - 1

dihedrals.loc[dihedrals[dihedrals["atom1"] >= 215-2].index, "atom1"] = dihedrals[dihedrals["atom1"] >= 215-2]["atom1"] - 1
dihedrals.loc[dihedrals[dihedrals["atom2"] >= 215-2].index, "atom2"] = dihedrals[dihedrals["atom2"] >= 215-2]["atom2"] - 1
dihedrals.loc[dihedrals[dihedrals["atom3"] >= 215-2].index, "atom3"] = dihedrals[dihedrals["atom3"] >= 215-2]["atom2"] - 1
dihedrals.loc[dihedrals[dihedrals["atom4"] >= 215-2].index, "atom4"] = dihedrals[dihedrals["atom4"] >= 215-2]["atom4"] - 1

dihedrals.loc[dihedrals.index,"dihedral id"] = list(range(1,len(dihedrals)+1)) 

In [29]:
# I am gonna ignore the non-used types, for now
dihedrals["dihedral type"].unique()

array([6, 4, 5, 2, 3])

In [35]:
def write_mol_file():
    f = open("IONOMER3_.mol", "w")
    f.write("# Created by Karina Chiñas Fuentes | Python | 22.09.24\n")
    f.write(f"\n{len(data)} atoms\n{len(bonds)} bonds\n"+
            f"{len(angles)} angles\n{len(dihedrals)} dihedrals")
    f.write("\n\n")
    f.write("Coords\n\n")
    for i in range(len(data)):
        f.write(f"{data['atom id'][i]}\t{data['x'][i]}"+
                f"\t{data['y'][i]}\t{data['z'][i]}\n")
    f.write("\nTypes\n\n")    
    f.write("Coords\n\n")
    for i in range(len(data)):
        f.write(f"{data['atom id'][i]}\t{data['element id'][i]}\n")
    f.write("\nCharges\n\n")
    for i in range(len(data)):
        f.write(f"{data['atom id'][i]}\t{data['charge'][i]}\n")
    f.write("\nBonds\n\n")
    for i in range(len(bonds)):
        f.write(f"{bonds['bond id'][i]}\t{bonds['bond type'][i]}\t"+
                f"{bonds['atom1'][i]}\t{bonds['atom2'][i]}\n")
    f.write("\nAngles\n\n")
    for i in range(len(angles)):
        f.write(f"{angles['angle id'][i]}\t{angles['angle type'][i]}\t"+
                f"{angles['atom1'][i]}\t{angles['atom2'][i]}\t{angles['atom3'][i]}\n")
    f.write("\nDihedrals\n\n")
    for i in range(len(dihedrals)):
        f.write(f"{dihedrals['dihedral id'][i]}\t{dihedrals['dihedral type'][i]}"+
                f"\t{dihedrals['atom1'][i]}\t{dihedrals['atom2'][i]}\t"+
                f"{dihedrals['atom3'][i]}\t{dihedrals['atom4'][i]}\n")

In [36]:
write_mol_file()
! code IONOMER3_.mol

In [38]:
#! touch ionomer_parms.lammps
! code ionomer_parms.lammps

In [42]:
#! code ATB/C65H4F128O15S3/parm.lammps

In [32]:
#!code NAFION_3repsPOLYMER.data ATB/C65H4F128O15S3/wNafion.mol

In [4]:
4**(1/12) == 2**(1/6)

True